In [1]:
import os
from glob import glob
from pathlib import Path
import shutil
import sys
import time

In [11]:
today = time.strftime('%Y-%m-%d')

In [6]:
def get_all_files(card_path):
    all_files = []
    for root, dirs, files in os.walk(card_path):
        for file in files:
            all_files.append(os.path.join(root, file))
    return all_files

In [6]:
# get paths of SD cards connected
if sys.platform == 'darwin':
    connected_sd_cards = [Path(card) for card in glob('/Volumes/Untitled*/')]
elif sys.platform == 'win32':
    connected_sd_cards = [Path(card) for card in glob('E:/')]
elif sys.platform == 'linux':
    connected_sd_cards = [Path(card) for card in glob('/media/')]
else:
    # throw error if platform is unknown
    raise OSError('Unknown platform')

In [7]:
card_id = {}
# card label and files to download
data_incoming = {''}
cam_id = 0
for card in connected_sd_cards:
    if any(['.360' in file for file in get_all_files(card)]):
        files = [file for file in get_all_files(card) if '.360' in file]
        card_id['360cam'] = {'card': card, 'files': files}
    elif any(['.mp4' in file.lower() for file in get_all_files(card)]):
        cam+=1
        files = [file for file in get_all_files(card) if '.mp4' in file.lower()]
        card_id[f'cam{cam}'] = {'card': card, 'files': files}
    elif any(['TRACK' in file for file in get_all_files(card)]):
        # TO DO: make sure audio files recorded on same date as video files
        files = [file for file in get_all_files(card) if 'TRACK' in file]
        card_id['microphones'] = {'card': card, 'files': files}
    else:
        print(f'Unknown card: {card} contains the following files:')
        print(get_all_files(card))

In [ ]:
assert len(card_id) == 4, 'Not all cards are connected'

In [8]:
# func for getting the creation date of a file
def get_creation_date(path_to_file):
    return time.strftime('%Y-%m-%d', time.gmtime(os.path.getctime(path_to_file)))

In [9]:
# make sure all files have the same creation date and print the files for each unique date detected
for card in card_id:
    dates = []
    for file in card_id[card]['files']:
        dates.append(get_creation_date(file))
    dates = list(set(dates))
    assert len(dates) == 1, f'Files on {card} were not all created on the same date'
    print(f'Files on {card} were created on {dates[0]}')

In [14]:
# ask user if ok to transfer the files
print('The following files will be transferred:')
for card in card_id:
    files_on_card = [os.path.basename(file) for file in card_id[card]['files']]
    print(f'{card}: {files_on_card}\n')
ok = input('Transfer files? (y/n): ')
if ok.lower() == 'y':
    pass
else:
    raise OSError('User aborted transfer')

The following files will be transferred:


''